# Get a list of hospitals in the Netherlands using Foursquare



In [48]:
from bs4 import BeautifulSoup
import requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if this is not installed yet
import folium # map rendering library


#### Define Foursquare Credentials and Version

In [49]:
CLIENT_ID = '51VTA3EKZVQ2FFWWAIXBKRRPIUPLWKY1ZAKTNQ54P0UL2CBO' # your Foursquare ID
CLIENT_SECRET = 'FBIX1RWOQTGEYPNHIRWPEKSYAHP5FULPD4Y23PDZ4UAN0E4M' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 51VTA3EKZVQ2FFWWAIXBKRRPIUPLWKY1ZAKTNQ54P0UL2CBO
CLIENT_SECRET:FBIX1RWOQTGEYPNHIRWPEKSYAHP5FULPD4Y23PDZ4UAN0E4M


#### Let's search for all Hospitals in the Netherlands.

Baarn is in the middle of the Neterlands according to
https://nl.wikipedia.org/wiki/Geografisch_middelpunt_van_Nederland

If you have a radius of 172 km it covers the whole of the Netherlands


In [50]:
#latitude = '52.962226'  #  Zeepromenade, Huisduinen, Den Helder
#longitude = '4.732374'  #  Zeepromenade, Huisduinen, Den Helder 
#latitude = '52.6324'  #  Alkmaar
#longitude = '4.7534'  #  Alkmaar
#latitude = '52.0798'  #  Woerden
#longitude = '4.8627'  #  Woerden
latitude = '52.216667'  #  Baarn is in the middle of the Netherlands
longitude = '5.3'  #  Baarn is in the middle of the Netherlands
radius = 172000 # define radius of 172 km which covers in a circle all of the Netherlands around Baarn
#radius = 30000 # define radius of 172 km which covers in a circle all of the Netherlands around Baarn



#### Now, let's get all Hospitals in the Netherlands.
Limit is set to 999 but that is just to get everything

Use Foursquare search. I am not interested in review etc so I do not need explore

In [51]:
#cat = '4bf58dd8d48988d104941735'  # Medical Centres
cat='4bf58dd8d48988d196941735'  # Hospital

LIMIT = 999 # limit of number of venues returned by Foursquare API
'''
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT,cat)
'''
# I use search instead of explore. I am not interested in reviews etc. Just the location
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}&categoryId={}'\
.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT,cat)


Send the GET request and examine the results

In [52]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d592a0f0d2be7002cde88ed'},
 'response': {'venues': [{'id': '4b4d8007f964a5202cd326e3',
    'name': 'HMC Westeinde',
    'location': {'address': 'Lijnbaan 32',
     'lat': 52.0738602,
     'lng': 4.3000598,
     'labeledLatLngs': [{'label': 'display',
       'lat': 52.0738602,
       'lng': 4.3000598}],
     'distance': 70132,
     'postalCode': '2512 VA',
     'cc': 'NL',
     'neighborhood': 'Centrum',
     'city': 'Den Haag',
     'state': 'Zuid-Holland',
     'country': 'Nederland',
     'formattedAddress': ['Lijnbaan 32', '2512 VA Den Haag', 'Nederland']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medical_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1566124559',
    'hasPerk': False},
   {'id': '4b8fb628f964a520535e33e3',
    'name': 'Flevozi

All the information is in the items key. I reuse the get_category_type function from the Foursquare lab.

In [53]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [54]:
venues = results['response']['venues']
    
nearby_venues = json_normalize(venues) # flatten JSON
# filter columns
filtered_columns = ['location.address','name','location.lat','location.lng','location.distance','location.formattedAddress']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues.to_csv(r'df_hospitals_Netherlands_unfiltered.csv')
print('{} locations were returned by Foursquare.'.format(nearby_venues.shape[0]))

#nearby_venues.dropna(subset=['location.address'],inplace=True) # only use data if address is available
nearby_venues.head(20)
hospitals=nearby_venues.copy()


50 locations were returned by Foursquare.


In [55]:
nearby_venues.head()

,location.address,name,location.lat,location.lng,location.distance,location.formattedAddress
0,Lijnbaan 32,HMC Westeinde,52.073860,4.300060,70132,"[Lijnbaan 32, 2512 VA Den Haag, Nederland]"
1,Hospitaalweg 1,Flevoziekenhuis Almere,52.369220,5.223330,17766,"[Hospitaalweg 1, 1315 RA Almere, Nederland]"
2,Heidelberglaan 25,Prinses Maxima Centrum,52.090273,5.183717,16156,"[Heidelberglaan 25, 3584 CS Utrecht, Nederland]"
3,Els Borst-Eilersplein 275,HagaZiekenhuis,52.055893,4.263276,73060,"[Els Borst-Eilersplein 275, 2545 AA Den Haag, ..."
4,Hilvarenbeekseweg 60,ETZ Elisabeth,51.539430,5.103520,76588,"[Hilvarenbeekseweg 60, 5022 GC Tilburg, Nederl..."


Filter on names containing a string which is common for hospitals to filter out incorrect data

Finally, let's visualize the hospitals with a radius of 7 km

In [56]:
# create map
latitude = 52.962226  #  Zeepromenade, Huisduinen, Den Helder
longitude = 4.732374  #  Zeepromenade, Huisduinen, Den Helder 
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
#x = np.arange(kclusters)
#ys = [i + x + (i*x)**2 for i in range(kclusters)]
#colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
# Use Circle since I want to show a radius in meters
# CircleMarker has radius in pixels
markers_colors = []
for lat, lon, poi in zip(hospitals['location.lat'], hospitals['location.lng'], hospitals['name'],):
    label = folium.Popup(str(poi) , parse_html=True)
    folium.Circle(
        [lat, lon],
        radius=7000,
        popup=label,
    #    color=rainbow[cluster-1],
        fill=True,
    #    fill_color=rainbow[cluster-1],
        fill_opacity=0.0).add_to(map_clusters)
  
       
map_clusters